# Walkthrough:

In [ ]:
from Analysis.gaitanalysis import Recording2D
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

%matplotlib inline

In [ ]:
rec_list = []
for file in Path.iterdir(Path("")):
    if file.suffix == ".csv":
        if not "filtered" in file.name:
            csv_file = str(file)
            rec = Recording2D(filepath=csv_file, recorded_framerate=30)
            if rec.full_df_from_hdf.shape[0] > 150:
                rec.run(
                    intrinsic_camera_calibration_filepath=""
                )
                rec.run_gait_analysis()
                rec_list.append(rec)

In [ ]:
tracking_overview = pd.DataFrame({})
for i, rec in enumerate(rec_list):
    for bodypart in rec.bodyparts:
        df = rec.bodyparts[bodypart].df_raw
        likelihood_threshold = 0.5
        marker_detected_per_total_frames = df.loc[df['likelihood']>likelihood_threshold, :].shape[0]/df.loc[:, :].shape[0]
        tracking_overview.loc[bodypart, i] = marker_detected_per_total_frames
tracking_overview.mean(axis=1)

In [ ]:
output_path = Path("")
groups = pd.read_excel("")

In [ ]:
for parameter in rec_list[0].parameters_over_steps:
    df = pd.DataFrame(data={}, columns=["Data", "group_id", "subject_id", "session_id"])
    for i, rec in enumerate(rec_list):
        data = np.NaN
        group_id = np.NaN
        animal = rec.metadata["animal"]
        paradigm = rec.metadata["paradigm"]
        try:
            group_id = groups.loc[groups["subject_id"] == animal, "group_id"].values[0]
        except:
            continue
        try:
            data = rec.parameters_paw_placed[parameter]
        except:
            pass
        if "ä" in paradigm:
            paradigm = paradigm.replace("ä", "e")
        if paradigm == "post6xIgG":
            paradigm = "post-6xIgG"
        if paradigm == "post3xIgG":
            paradigm = "post-3xIgG"
        if paradigm == "post1xIgG":
            paradigm = "post-1xIgG"
        if paradigm == "post-OP3":
            paradigm = "pre-OP3"
        df.loc[i, "subject_id"] = animal
        df.loc[i, "session_id"] = paradigm
        df.loc[i, "group_id"] = group_id
        df.loc[i, "Data"] = data

    df.dropna(inplace=True)
    df.to_csv(output_path.joinpath(parameter + "_paw_placed.csv"), index=False)

In [ ]:
for file in output_path.iterdir():
    if (file.suffix == ".csv") and "paw_placed" in file.name:
        print(file.name)
        plotting = pd.read_csv(output_path.joinpath(file))
        fig = plt.figure(figsize=(10, 5))
        sns.pointplot(
            data=plotting,
            y="Data",
            x="session_id",
            hue="group_id",
            order=[
                "pre-OP1",
                "pre-OP2",
                "pre-OP3",
                "post-OP",
                "post-1xIgG",
                "post-3xIgG",
                "post-6xIgG",
            ],
        )
        plt.show()

In [ ]:
output_path = Path("")

In [ ]:
for parameter in rec_list[0].parameters_over_steps:
    df = pd.DataFrame(data={}, columns=["Data", "group_id", "subject_id", "session_id"])
    for i, rec in enumerate(rec_list):
        animal = rec.metadata["animal"]
        paradigm = rec.metadata["paradigm"]
        try:
            group_id = groups.loc[groups["subject_id"] == animal, "group_id"].values[0]
        except:
            continue
        try:
            data = rec.parameters_paw_placed[parameter]
        except:
            pass
        if "ä" in paradigm:
            paradigm = paradigm.replace("ä", "e")
        if paradigm == "post6xIgG":
            paradigm = "post-6xIgG"
        if paradigm == "post3xIgG":
            paradigm = "post-3xIgG"
        if paradigm == "post1xIgG":
            paradigm = "post-1xIgG"
        if paradigm == "post-OP3":
            paradigm = "pre-OP3"
        df.loc[i, "subject_id"] = animal
        df.loc[i, "session_id"] = paradigm
        df.loc[i, "group_id"] = group_id
        for n in range(-7, 7):
            try:
                df.loc[i, n] = rec.parameters_as_psth[parameter][n]
            except:
                pass
    df.to_csv(output_path.joinpath(parameter + "_psth.csv"), index=False)